In [2]:
import random
import time
import pandas as pd
from vexa_client import VexaClient

In [3]:
import os
TOKEN = os.getenv('ADMIN_API_TOKEN')

In [4]:
url = "http://localhost:8056"

In [5]:
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

In [6]:

# Config
num_users = 3
base_url = "http://localhost:8056"
meeting_url = "https://meet.google.com/nny-dera-dge"
native_meeting_id = meeting_url.split("/")[-1]
platform = "google_meet"

# Create multiple users
users = []
clients = []

for i in range(num_users):
    new_user = admin_client.create_user(
        email=f"multi_user_{random.randint(1, 1000000)}@example.com", 
        name=f"TestUser_{i+1}"
    )
    user_id = new_user['id']
    users.append(new_user)
    
    token_info = admin_client.create_token(user_id=user_id)
    user_api_key = token_info['token']
    
    client = VexaClient(
        base_url=base_url,
        api_key=user_api_key
    )
    clients.append(client)
    
    print(f"Created user {i+1}/{num_users} with ID {user_id}")

# Send all bots to meeting
meetings = []

for i, client in enumerate(clients):
    meeting_info = client.request_bot(
        platform=platform,
        native_meeting_id=native_meeting_id,
        bot_name=f"Bot_{i+1}"
    )
    meetings.append(meeting_info)
    meeting_id = meeting_info['id']
    print(f"Bot {i+1}/{num_users} sent to meeting with ID {meeting_id}")

# Wait for bots to join
print("Waiting 15 seconds for bots to join the meeting...")
time.sleep(15)


DEBUG: Making POST request to http://localhost:8056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: {'email': 'multi_user_697255@example.com', 'name': 'TestUser_1'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 16 Apr 2025 07:43:15 GMT, Wed, 16 Apr 2025 07:43:16 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '128', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"multi_user_697255@example.com","name":"TestUser_1","image_url":null,"id":59,"created_at":"2025-04-16T07:43:16.563291"}...

DEBUG: Making POST request to http://localhost:8056/admin/users/59/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 16 Apr 2025 07:43:15 GMT, Wed, 16 Apr 2025 07:43:16 GMT', 'server': 'uvicorn, uvicorn', 'content-lengt

In [39]:

# Monitor transcripts
all_transcripts = {}

for i, client in enumerate(clients):
    try:
        transcript = client.get_transcript(native_meeting_id=native_meeting_id, platform=platform)
        segments = transcript.get('segments', [])
        df = pd.DataFrame(segments).sort_values(['created_at','start']).drop_duplicates('start',keep='last')
        all_transcripts[f"User_{i+1}"] = df
        print(f"Retrieved transcript for User_{i+1} with {len(segments)} segments")
    except Exception as e:
        print(f"Error retrieving transcript for User_{i+1}: {e}")

# Display first transcript
if all_transcripts and len(all_transcripts) > 0:
    first_user = list(all_transcripts.keys())[0]
    print(f"\nTranscript from {first_user}:")
    display(all_transcripts[first_user].tail(10))

# Compare transcripts
if len(all_transcripts) >= 2:
    all_texts = set()
    for user, df in all_transcripts.items():
        all_texts.update(df['text'].tolist())
    
    comparison = {}
    for text in all_texts:
        comparison[text] = {}
        for user, df in all_transcripts.items():
            comparison[text][user] = text in df['text'].values
    
    comparison_df = pd.DataFrame(comparison).T
  #  print("\nTranscript comparison across users:")
    display(comparison_df)


DEBUG: Making GET request to http://localhost:8056/transcripts/google_meet/nny-dera-dge
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'HDVYW6AWoHHJa6DFnrz6yiErc6ye0RcgUgivy48G'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Wed, 16 Apr 2025 07:51:26 GMT, Wed, 16 Apr 2025 07:51:26 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '3304', 'content-type': 'application/json'}
DEBUG: Response content: {"id":86,"platform":"google_meet","native_meeting_id":"nny-dera-dge","constructed_meeting_url":"https://meet.google.com/nny-dera-dge","status":"active","start_time":"2025-04-16T07:43:16.882323","end_time":null,"segments":[{"start":80.724,"end":85.724,"text":" So, now I will quickly open it right away.","language":"en","created_at":null,"speaker":null},{"start":90.0,"end":92.0,"text":" That's it. How much do you want?","language":"en","created_at":null,"speaker":null},{"start":92.0,"end":98.0,"te...
Retrieved t

,start,end,text,language,created_at,speaker
14,253.800,254.500,It's all in the car.,en,None,None
15,254.500,260.200,"No, you understand that they looked at me in ...",en,None,None
16,275.761,283.733,"I have a headache because of this, you know, ...",en,None,None
17,296.286,303.286,It didn't get any worse. I don't know what's ...,en,None,None
18,303.286,305.286,I don't know.,en,None,None
19,305.286,307.286,Mom! Mom!,en,None,None
20,307.286,309.286,"Yes, my dear.",en,None,None
21,309.286,311.286,Thank you very much.,en,None,None
22,311.286,313.286,I understood.,en,None,None
23,313.286,315.286,I was lying down.,en,None,None


,User_1,User_2
"So, now I will quickly open it right away.",True,False
I'll think about it.,False,True
Maybe one more...,False,True
I understand you.,True,False
"Now, now, wait a second, okay?",False,True
"Well, let's go. What's the difference between us?",False,True
Let's try it.,False,True
Thank you very much for listening to this video.,False,True
"Let's wait a little bit more, we'll try, maybe we'll meet somewhere else.",True,False
He sent me to the court yesterday.,True,False
